In [36]:
import pandas as pd
import time
import pandas as pd
from IPython.display import display
pd.options.display.max_rows = None
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
df = pd.read_csv('./trips.csv')

In [38]:
df.head()

,trip_id,parsing_date,is_direct_route,title,published_date,visits,price,passengers_free,passengers_max,departure_time,driver_name,driver_age,driver_status,car,driver_num_trips,driver_stars,comments_count,route
0,556087497,4.8:14:31,1,"Воронеж,Старый Оскол",04/08/20,38,240,1,3,2016-08-04 15:00:00,Сергей Г,29,Уверенный,Lada Granta,4,5,1,voronezh_staryi-oskol
1,555490062,4.8:14:31,0,"Воронеж,Старый Оскол,Белгород",03/08/20,102,180,3,3,2016-08-04 16:00:00,Александр М,23,NaN,NaN,2,NaN,0,voronezh_staryi-oskol
2,555130382,4.8:14:31,1,"Воронеж,Старый Оскол",03/08/20,38,240,Заполнен,3,2016-08-04 16:30:00,Владислав Р,20,NaN,Daewoo Nexia,4,NaN,0,voronezh_staryi-oskol
3,554134942,4.8:14:31,0,"Самара,Воронеж,Старый Оскол,Ракитное",02/08/20,158,180,Заполнен,-1,2016-08-04 16:40:00,Владимир М,47,Уверенный,Suzuki Grand vitara,2,5,2,voronezh_staryi-oskol
4,555790137,4.8:14:31,1,"Воронеж,Старый Оскол",03/08/20,24,240,2,3,2016-08-04 17:00:00,Михаил Л,29,Уверенный,ваз Priora,9,5,2,voronezh_staryi-oskol


In [39]:
len(df.trip_id.unique())

100

In [40]:
def handle_dates():
    df.departure_time = pd.to_datetime(df.departure_time)
    df.published_date = pd.to_datetime(df.published_date, dayfirst=True)
    df.parsing_date = pd.to_datetime(df.parsing_date, format='%d.%m:%H:%M')
    df.parsing_date = df.parsing_date.apply(lambda x: x.replace(year=2016))
    df.published_date = df.published_date.apply(lambda x: x.replace(year=2016))
handle_dates()

In [41]:
df.head()

,trip_id,parsing_date,is_direct_route,title,published_date,visits,price,passengers_free,passengers_max,departure_time,driver_name,driver_age,driver_status,car,driver_num_trips,driver_stars,comments_count,route
0,556087497,2016-08-04 14:31:00,1,"Воронеж,Старый Оскол",2016-08-04,38,240,1,3,2016-08-04 15:00:00,Сергей Г,29,Уверенный,Lada Granta,4,5,1,voronezh_staryi-oskol
1,555490062,2016-08-04 14:31:00,0,"Воронеж,Старый Оскол,Белгород",2016-08-03,102,180,3,3,2016-08-04 16:00:00,Александр М,23,NaN,NaN,2,NaN,0,voronezh_staryi-oskol
2,555130382,2016-08-04 14:31:00,1,"Воронеж,Старый Оскол",2016-08-03,38,240,Заполнен,3,2016-08-04 16:30:00,Владислав Р,20,NaN,Daewoo Nexia,4,NaN,0,voronezh_staryi-oskol
3,554134942,2016-08-04 14:31:00,0,"Самара,Воронеж,Старый Оскол,Ракитное",2016-08-02,158,180,Заполнен,-1,2016-08-04 16:40:00,Владимир М,47,Уверенный,Suzuki Grand vitara,2,5,2,voronezh_staryi-oskol
4,555790137,2016-08-04 14:31:00,1,"Воронеж,Старый Оскол",2016-08-03,24,240,2,3,2016-08-04 17:00:00,Михаил Л,29,Уверенный,ваз Priora,9,5,2,voronezh_staryi-oskol


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207 entries, 0 to 206
Data columns (total 18 columns):
trip_id             207 non-null int64
parsing_date        207 non-null datetime64[ns]
is_direct_route     207 non-null int64
title               207 non-null object
published_date      207 non-null datetime64[ns]
visits              207 non-null int64
price               207 non-null int64
passengers_free     207 non-null object
passengers_max      207 non-null int64
departure_time      207 non-null datetime64[ns]
driver_name         207 non-null object
driver_age          207 non-null int64
driver_status       140 non-null object
car                 198 non-null object
driver_num_trips    207 non-null int64
driver_stars        134 non-null float64
comments_count      207 non-null int64
route               207 non-null object
dtypes: datetime64[ns](3), float64(1), int64(8), object(6)
memory usage: 30.7+ KB


In [43]:
df[df.trip_id == [df.trip_id.unique()[105]]].sort_values('parsing_date')

IndexError: index 105 is out of bounds for axis 0 with size 100

In [44]:
def get_trips_df():
    df_grouped = df.groupby('trip_id')
    parse_min = df_grouped.agg({'parsing_date': np.min})
    parse_max = df_grouped.agg({'parsing_date': np.max})
    wright_ids = []
    for id in parse_min.index:
        if df.parsing_date.min() < df[df.trip_id == id]['published_date'].iloc[0] and\
           df.parsing_date.max() > df[df.trip_id == id]['departure_time'].iloc[0]:
            wright_ids.append(id)
    # for id in wright_ids:
    clear_df = df[df.trip_id.isin(wright_ids)].drop_duplicates(['trip_id'], keep='last')
    first_parsing_date = []
    clear_df.apply(lambda row: first_parsing_date.append(parse_min.loc[row['trip_id']]['parsing_date']), axis=1)
    first_parsing_date = pd.Series(first_parsing_date, index=clear_df.index)
    clear_df['first_parsing_date'] = first_parsing_date
    return clear_df
clear_df = get_trips_df()

In [50]:
clear_df.parsing_date.min()

Timestamp('2016-08-04 14:31:00')

In [47]:
clear_df.shape
#clear_df.columns

(0, 19)

In [48]:
clear_df.parsing_date.max() - clear_df.parsing_date.min()

NaT

In [49]:
clear_df.parsing_date.min(), clear_df.parsing_date.max()

(NaT, NaT)

In [34]:
clear_df[['departure_time', 'published_date', 'first_parsing_date', 'price', 'passengers_free', 'title']].head(100)

,departure_time,published_date,first_parsing_date,price,passengers_free,title
3226,2016-06-29 11:00:00,2016-06-29,2016-06-29 07:02:00,250,3,"Воронеж,Старый Оскол"
3377,2016-06-29 11:30:00,2016-06-29,2016-06-29 11:04:00,250,3,"Старый Оскол,Воронеж"
3482,2016-06-29 12:30:00,2016-06-29,2016-06-29 09:05:00,150,2,"Старый Оскол,Воронеж"
3493,2016-06-29 14:00:00,2016-06-29,2016-06-29 13:03:00,200,2,"Воронеж,Старый Оскол"
3998,2016-06-29 19:10:00,2016-06-29,2016-06-29 16:02:00,200,3,"Воронеж,Старый Оскол"
4043,2016-06-29 18:30:00,2016-06-29,2016-06-29 18:04:00,200,3,"Старый Оскол,Воронеж"
4117,2016-06-29 19:30:00,2016-06-29,2016-06-29 18:02:00,150,2,"Воронеж,Старый Оскол"
4164,2016-06-29 19:20:00,2016-06-29,2016-06-29 15:04:00,100,3,"Старооскольское городское кладбище, Р189, Белг..."
4165,2016-06-29 19:30:00,2016-06-29,2016-06-29 16:05:00,200,3,"Старый Оскол,Воронеж"
4166,2016-06-29 20:00:00,2016-06-29,2016-06-29 15:04:00,200,2,"Старый Оскол,Воронеж"


In [35]:
clear_df.title

3226                                  Воронеж,Старый Оскол
3377                                  Старый Оскол,Воронеж
3482                                  Старый Оскол,Воронеж
3493                                  Воронеж,Старый Оскол
3998                                  Воронеж,Старый Оскол
4043                                  Старый Оскол,Воронеж
4117                                  Воронеж,Старый Оскол
4164     Старооскольское городское кладбище, Р189, Белг...
4165                                  Старый Оскол,Воронеж
4166                                  Старый Оскол,Воронеж
4167                                  Старый Оскол,Воронеж
4401                                  Старый Оскол,Воронеж
4692                          Старый Оскол,Воронеж,Королев
5187                           Старый Оскол,Воронеж,Москва
5247                                  Воронеж,Старый Оскол
5404                                  Старый Оскол,Воронеж
5405                                  Старый Оскол,Ворон